#Mount Google Drive and unzip data

In [1]:
#mounting google drive (data are placed in root in zip file)

from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


In [2]:
!unzip -q "/content/drive/MyDrive/places_64x64_new_correct.zip"

replace out_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
N


# Import:


*   own developed modules from Google Drive
*   standard libs and 3rd party libs


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/modules')

import os
print(os.listdir('/content/drive/MyDrive/modules'))

from dataread import DataRead

#from custom_losses import generatorMSELossInJoint, generatorAdversarialLoss, discriminatorAdversarialLoss
#from plotter import generateAndPlotImgs, plotImages
from ganscape import GANScape
from train_test import trainNetwork, testNetwork
from plotter import generateAndPlotImgs

['__pycache__', 'custom_losses.py', 'dataread.py', 'plotter.py', 'network_structure.py', 'train_test.py', 'ganscape.py']


In [3]:
from __future__ import print_function, division

from keras.layers import Input, Activation, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise, Lambda
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Concatenate
from keras.layers.advanced_activations import ReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam, Adadelta
from keras import losses, activations
from keras.callbacks import TensorBoard
import keras.backend as K
from keras.utils.vis_utils import plot_model
import scipy
import tensorflow as tf
from IPython.display import clear_output
from datetime import datetime

from PIL import Image  
import PIL  
import matplotlib.pyplot as plt

import os
import time
import numpy as np
import copy

# Set parameters

In [4]:
#init some paramters

#--------------------------------------------------------CAN BE MODIFIED-------------------------------------
epochs = 20
batchSize = 128

generatorDescriber = {
    "earlyPart":{
        "numOfBlocks": 5,
        "filterNums": [64, 128, 128, 256, 256],
        "kernelSizes": [5, 3, 3, 3, 3],
        "strideSizes": [1, 2, 1, 2, 1]
    },
    "midPart":{
        "numOfBlocksWithDilation": 4,
        "filterNum": 256
    },
    "endPart":{
        "numOfBlocks": 4,
        "convolutionType": ["transposed", "normal", "transposed", "normal"],
        "filterNums": [128, 128, 64, 32],
        "kernelSizes": [4, 3, 4, 3],
        "strideSizes": [2, 1, 2, 1]
    }
}

discriminatorDescriber = {
    "global":{
        "numOfBlocks": 5,
        "filterNums": [64, 128, 256, 512, 512],
        "kernelSizes": [5, 5, 5, 5, 5],
        "strideSizes": [2, 2, 2, 2, 2]
    },
    "local":{
        "numOfBlocks": 4,
        "filterNums": [64, 128, 256, 512],
        "kernelSizes": [5, 5, 5, 5],
        "strideSizes": [2, 2, 2, 2]
    },
    "useGlobalAvgPool": False 
}

earlyStopPhases = {
    "phaseTwo": {
        "minForDiscAdv": 0.8,   #if [this value > disc_adv_loss], then skip remained part of phase II
    },
    "phaseThree": {
        "minForDiscAdv": 0.8,  #if [this value > disc_adv_loss] OR
        "maxForGenAdv": 100    #   [this value < gen_adv_loss] then skip remained part of phase III
    }
}

optimizers = {
    "onlyGenMse": Adam(learning_rate=0.01),
    "onlyDisc": Adam(learning_rate=0.001),
    "onlyGenJoint": Adam(learning_rate=0.001)
}
weightsForJointLoss = {
    "mse": 1,
    "adversarial": 1
}


ratios = {           # epoch start --> t = 1
    "phaseOne": 0.33, #    phase I -->   train generator with mse until: [t <= (phaseOne * total_step_num)]
    "phaseTwo": 0.33  #    phase II -->  train discriminator with adversarial until: [t <= ((phaseOne + phaseTwo) * total_step_num)]
}                    #    phase III --> train generator with joint and discriminator with adversarial until end of epoch

plotImgsDict = {
    "isUsed": True,          
                             # if False then "plot-imgs" feature is not used, so all of other propertys of this object dont matter            
    "saveImgsToFile": True, 
                             # if True then images are saved, if False then images are only shown
    "sleepingTime": 5, 
                             # time of showing images during training - next epoch starts when expired
    "numOfImgs": 6,          
                             # range of accepted value [1, batchSize]
    "numOfImgMapsToSave": 5,

    "figSize": (15,15)       
                             # size of figure
}

whatToSave = "all"           #options: 'all' (means save model in every epoch) or 'best' (means only the model with min valid_loss [mse and joint generator loss] will be saved)

numOfBatchesLoadedAtOnce = 10

paths = {
    "data": r"./",
    "logDirForTensorboard": '/content/drive/MyDrive/logs',
    "saveModelDir": '/content/drive/MyDrive/saved_models',
    "saveImgsDirDuringTrain": '/content/drive/MyDrive/saved_imgs_train',
    "saveImgsDirDuringTest": '/content/drive/MyDrive/saved_imgs_test',
}


#--------------------------------------------------------NOT SHOULD BE MODIFIED------------------------------------------------------

numOfAllDataSample = {
    "train": 172703,
    "valid": 20318,
    "test": 10159
}
stepsPerEpoch = {
    "train": numOfAllDataSample["train"] // batchSize,
    "valid": numOfAllDataSample["valid"] // batchSize
}
stepsForTest =numOfAllDataSample["test"] // batchSize

print("Steps per epoch:", stepsPerEpoch)

Steps per epoch: {'train': 1349, 'valid': 158}


#Runnable scripts for train and test


*   set parameters
*   build network
*   train & validation
*   test



In [5]:
#Container-handler for train, valid, test data

#trainData = DataRead(paths["data"],'train', batchSize, numOfBatchesLoadedAtOnce, shuffle = True)
#validData = DataRead(paths["data"],'valid', batchSize, numOfBatchesLoadedAtOnce, shuffle = True)
#testData = DataRead(paths["data"],'test', batchSize, numOfBatchesLoadedAtOnce, shuffle = True)

In [6]:
#create network
net = GANScape(batchSize=batchSize,
               inputImgShape = (64,64,3),
               localDiscInputShape = (28,28,3),
               generatorDescriber = generatorDescriber,
               discriminatorDescriber = discriminatorDescriber,
               optimizers = optimizers,
               weightsForJointLoss = weightsForJointLoss)


#-------------------------------- use this if you want to train the model from 0 ------------------------------------------------ 
net.buildAndCompile(loadModels=False)
#--------------------------------------------------------------------------------------------------------------------------------

#-------------------------------- use this if you want to load a model and train it further ------------------------------------
#net.buildAndCompile(loadModels=True, loadPath='/content/drive/MyDrive/saved_models/2020_12_12-12_12_08/epoch_1')
#---------------------------------------------------------------------------------

In [7]:
#training
trainLoss, validLoss = trainNetwork(net = net,
                             epochs = epochs,
                             batchSize = batchSize,
                             stepsPerEpoch = stepsPerEpoch,
                             ratios = ratios,
                             numOfBatchesLoadedAtOnce = numOfBatchesLoadedAtOnce,
                             plotImgsDict = plotImgsDict,
                             earlyStopPhases = earlyStopPhases,
                             whatToSave = whatToSave, 
                             paths = paths)

Training with 
  -epochs=20,
  -batchSize=128
  -steps for training=1349,
  -steps for validation=158,
  -phase I until step=445,
 -phase II until step=890


172703
172703
172703
20318
20318
20318


KeyboardInterrupt: ignored

In [8]:

#test
testLoss = testNetwork(net = net, 
                       batchSize = batchSize, 
                       stepsForTest = stepsForTest, 
                       numOfBatchesLoadedAtOnce = numOfBatchesLoadedAtOnce, 
                       plotImgsDict = plotImgsDict, 
                       paths = paths)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
%load_ext tensorboard

In [ ]:

%tensorboard --logdir /content/drive/MyDrive/logs

# Runnable scripts for load model and prediction

In [12]:
#set some parameters

#--------------------------------------------------------CAN BE MODIFIED------------------------------------------------------

pathForLoadModel = '/content/drive/MyDrive/saved_models/2020_12_12-12_12_08/epoch_1' #from here will the model loaded

pathToSaveImgMaps = '/content/drive/MyDrive/predicted_img_maps' #predicted img maps will saved here

numOfImgMaps = 5 #number of img maps to show (here in console)





#--------------------------------------------------------NOT SHOULD BE MODIFIED------------------------------------------------------
plotImgsDictForPredict = copy.deepcopy(plotImgsDict)
plotImgsDictForPredict["numOfImgMapsToSave"] = numOfImgMaps 

if not os.path.exists(pathToSaveImgMaps):
  os.makedirs(pathToSaveImgMaps)

In [9]:
#loading the model
net = GANScape(batchSize=batchSize,
               inputImgShape = (64,64,3),
               localDiscInputShape = (28,28,3),
               generatorDescriber = generatorDescriber,
               discriminatorDescriber = discriminatorDescriber,
               optimizers = optimizers,
               weightsForJointLoss = weightsForJointLoss)

net.buildAndCompile(loadModels=True, loadPath=pathForLoadModel)

In [10]:
#load test data
testData = DataRead(paths["data"], 'test', batchSize, numOfBatchesLoadedAtOnce, shuffle = True)

10159
10159
10159


In [16]:
#predict + show & save img map
imgMaps = generateAndPlotImgs(net, testData, plotImgsDictForPredict, numOfBatchesLoadedAtOnce, True)

[Image.fromarray(map.astype('uint8')).save(pathToSaveImgMaps+f'/{i}_IMG_MAP.jpg') for i, map in enumerate(imgMaps)]

Output hidden; open in https://colab.research.google.com to view.